# Assessing Vocal Measurements to Predict Parkinson’s Disease

Group 6 proposal: Naomi Choong, Yashas Ganesh Prasad, Braydon Newman, Marylyn Yuwono

# Introduction: 
Parkinson’s Disease (PD), a progressive neurodegenerative disorder impairing motor abilities, is characterized by tremors, stiffness, and slowed movements. Many individuals with PD also experience voice problems including abnormal voice quality, volume and pitch (Ma et al., 2020). Evidence shows vocal changes manifest during the prodromal phase of PD, prior to onset of clinical symptoms and diagnosis (Harel et al., 2004). This suggests that vocal measurements may be a potential biomarker to detect early stage PD. Early detection and treatment can slow disease progression and may help mitigate symptoms. 

To investigate potential vocal biomarkers of PD, we aim to answer the following question: **Can acoustic measurements of voice predict Parkinson’s disease based on voice recordings?**

We will analyze a dataset of various acoustic measurements from vocal recordings of 31 subjects. Data was collected by Little et al. (2009). An average of 6 recordings of sustained vowel phonation were taken for each subject.  Each row in the dataset corresponds to one voice recording from a subject.

Source: https://www.kaggle.com/datasets/gargmanas/parkinsonsdataset

## Variables
To examine the predictive potential of acoustic measurements, we will use the K-nearest neighbors classification algorithm. Variables were selected based on past literature, showing patients with PD had higher jitter, shimmer, and pitch variability, and reduced harmonics to noise ratio compared to healthy subjects (Jiménez-Jiménez et al., 1997; Ma et al., 2020). 


Variables used:
- `status` - health status of subject; Parkinson’s = 1, healthy =0
- `MDVP:Jitter (abs)` - absolute jitter in microseconds (variation in frequency)
- `Shimmer:DDA` - local shimmer in decibels (variation in amplitude)
- `PPE` - pitch period entropy (variation in pitch)
- `HNR` - harmonics-to-noise ratio (quantifies amount of additive noise in voice)


# Read and Tidy Data
... 


In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
install.packages("kknn")
library(kknn)

In [ ]:
# reading dataset using URL from GitHub
set.seed(1234)

url <- "https://raw.githubusercontent.com/naomichoong/ds_project/main/parkinsons.csv"
parkinsons_data <- read_csv(url)
parkinsons_data

In [ ]:
# Tidying and modifying parkinsons dataset

parkinsons_data <- parkinsons_data |>
    select(status, "MDVP:Jitter(Abs)", "Shimmer:DDA", PPE, HNR) |>
    rename(jitter = "MDVP:Jitter(Abs)", shimmer = "Shimmer:DDA", ppe = PPE, hnr = HNR) |>
    mutate(status = as_factor(status)) |>
    mutate(status = fct_recode(status, "Parkinson's" = "1", "Healthy" = "0"))
parkinsons_data


# Summarize data
... 



In [ ]:
# Using intital split to create training and testing datasets 

parkinsons_split <- initial_split(parkinsons_data, prop = 0.75, strata = status)
parkinsons_train <- training(parkinsons_split)
parkinsons_test <- testing(parkinsons_split)


In [ ]:
# summary statistics of training data

parkinsons_count <- parkinsons_train |>
    group_by(status) |>
    summarise(count = n())

parkinsons_count

parkinsons_mean <- parkinsons_train |>
    select(jitter, shimmer, ppe, hnr) |>
    map_df(mean)

parkinsons_mean

comparison_mean <- parkinsons_train |>
    group_by(status) |>
    summarize(mean_jitter = mean(jitter), 
           mean_shimmer = mean(shimmer), 
           mean_ppe = mean(ppe), 
           mean_hnr = mean(hnr)) 

comparison_mean

parkinsons_missing <- sum(is.na(parkinsons_train))
parkinsons_missing


# Exploratory Data Visualization
...



In [ ]:
# preliminary data visualization - exploratory data analysis of training data

options(repr.plot.width = 5, repr.plot.height = 4)
jitter_plot <- parkinsons_train |>
    ggplot(aes(x = jitter, fill = status)) +
    geom_histogram() +
    facet_grid(rows = vars(status)) +
    labs(x = "Absolute Jitter (microseconds)",
         y = "Number of observations",
         fill = "Diagnosis") +
    scale_x_continuous(limits = c(0, 0.0003)) +
    ggtitle("Fig 1. Distribution of Jitter by Diagnosis") +
    theme(text = element_text(size = 12))

jitter_plot

shimmer_plot <- parkinsons_train |>
    ggplot(aes(x = shimmer, fill = status)) +
    geom_histogram() +
    facet_grid(rows = vars(status)) +
    labs(x = "Local Shimmer (decibels)",
         y = "Number of Observations",
         fill = "Diagnosis") +
    scale_x_continuous(limits = c(0, 0.20)) +
    ggtitle("Fig 2. Distribution of Shimmer by Diagnosis") +
    theme(text = element_text(size = 12))

shimmer_plot

ppe_plot <- parkinsons_train |>
    ggplot(aes(x = ppe, fill = status)) +
    geom_histogram() +
    facet_grid(rows = vars(status)) +
    labs(x = "Pitch Period Entropy",
         y = "Number of Observations",
         fill = "Diagnosis") +
        scale_x_continuous(limits = c(0, 0.6)) +
    ggtitle("Fig 3. Distribution of Pitch Period Entrophy \n by Diagnosis") +
    theme(text = element_text(size = 12))

ppe_plot

hnr_plot <- parkinsons_train |>
    ggplot(aes(x = hnr, fill = status)) +
    geom_histogram() +
    facet_grid(rows = vars(status)) +
    labs(x = "Harmonics-to-Noise Ratio",
         y = "Number of Observations",
         fill = "Diagnosis") +
    scale_x_continuous(limits = c(0, 40)) +
    ggtitle("Fig 4. Distribution of Harmonics-to-Noise \n Ratio by Diagnosis") +
    theme(text = element_text(size = 12))

hnr_plot

## Preliminary Data Analysis Results
...

# K-Nearest Neighbours Classification Analysis
To investigate our question, the dataset was split into a training and testing set. The optimal k value was selected using a 5-fold cross validation on the training set. The k value with the highest accuracy estimate was used. The final model will be built using the optimal k value and used to evaluate the accuracy of our classifier on the test set. By computing and evaluating metrics for our classification model, we will be able to examine whether acoustic measures of voice can predict Parkinson’s disease. 

A line plot showing the accuracy estimate against the number of nearest neighbors (k) will be used to visualize results. Each point represents the accuracy of the K-nearest neighbors algorithm at a specific value of k, which helps determine the optimal value of k for our classification model. The final stacked bar plot will represent the health status of the subject by the predicted diagnosis of the subjects to evaluate the strength of our classifier. 

In [ ]:
# K-nearest neighbours classification analysis
set.seed(1234)
options(repr.plot.height = 7, repr.plot.width = 7)

# Create recipe to standardize training data
parkinsons_recipe <- recipe(status ~., data = parkinsons_train) |>
    step_center(all_predictors()) |>
    step_scale(all_predictors())

# Create nearest_neighbor model specification with tuning on number of neighbours
parkinsons_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

# 5-fold cross-validation on training data
parkinsons_vfold <- vfold_cv(parkinsons_train, v = 5, strata = status)

# Create workflow with k ranging from 1 to 50 (by 2) and collect metrics
gridvals <- tibble(neighbors = seq(from = 1, to = 50, by = 2))

parkinsons_results <- workflow() |>
    add_recipe(parkinsons_recipe) |>
    add_model(parkinsons_spec) |>
    tune_grid(resamples = parkinsons_vfold, grid = gridvals) |>
    collect_metrics()

accuracies <- parkinsons_results |>
    filter(.metric == "accuracy")
accuracies

# plot k (number of neighbours) vs accuracy to choose best k
parkinsons_k_plot <- accuracies |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    xlab("Number of Neighbours (k)") +
    ylab("Accuracy Estimate") +
    scale_x_continuous(breaks = seq(0, 50, by = 2)) +
    ggtitle("Fig 5. Estimated Accuracy versus Number of Neighbours") +
    theme(text = element_text(size = 14))

parkinsons_k_plot


In [ ]:
# knn classification final model with optimal k 

parkinsons_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 7) |>
    set_engine("kknn") |>
    set_mode("classification")

parkinsons_fit <- workflow() |>
    add_recipe(parkinsons_recipe) |>
    add_model(parkinsons_spec) |>
    fit(data = parkinsons_train)

# use final model to predict on test data
parkinsons_predictions <- predict(parkinsons_fit, parkinsons_test) |>
    bind_cols(parkinsons_test)

parkinsons_metrics <- parkinsons_predictions |>
    metrics(truth = status, estimate = .pred_class) |>
    filter(.metric == "accuracy")
parkinsons_metrics

parkinsons_conf_matrix <- parkinsons_predictions |>
    conf_mat(truth = status, estimate = .pred_class)
parkinsons_conf_matrix

# Final Visualization
.Naomi

In [ ]:
# Final Stacked Bar Graph  
predictions_count <- parkinsons_predictions |>
    semi_join(parkinsons_test) |>
    group_by(status, .pred_class) |>
    summarize(n=n())

predictions_count


parkinsons_final_graph <- predictions_count |>
   ggplot(aes(x = status, y = n, fill = .pred_class)) +
    geom_bar(position = "fill", stat = "identity") +
    labs(x = "Health Status of Subject", y = "Proportion Predicted", fill = "Predicted Health Status") +
    ggtitle("Fig 6. True health status of subject by predict status") +
    theme(text = element_text(size = 14))
parkinsons_final_graph
    

# Discussion

It is expected that the acoustic features will demonstrate significant differences between individuals with PD and healthy patients, providing a basis for classification which can be used for early detection of PD.

If our classification model shows good accuracy, voice analysis could become an early diagnosis screening tool and help initiate earlier treatment, potentially slowing disease progression. In the larger scope of medicine, the study’s findings can generate awareness regarding the utility of acoustic measurements for neurological and neurodegenerative disorders.

Future research could focus on larger, more diverse, patient populations. The study further provokes interest in other diagnostic tools and acoustic biomarkers that could be included to create more comprehensive diagnostic models. 

## References

Harel, B., Cannizzaro, M., & Snyder, P. J. (2004). Variability in fundamental frequency during speech in prodromal and incipient parkinson's disease: A longitudinal case study. Brain and Cognition, 56(1), 24-29. https://doi.org/10.1016/j.bandc.2004.05.002

Jiménez-Jiménez, F. J., Gamboa, J., Nieto, A., Guerrero, J., Orti-Pareja, M., Molina, J. A., García-Albea, E., & Cobeta, I. (1997). Acoustic voice analysis in untreated patients with parkinson's disease. Parkinsonism & Related Disorders, 3(2), 111-116. https://doi.org/10.1016/S1353-8020(97)00007-2

Little, M. A., McSharry, P. E., Hunter, E. J., Spielman, J., & Ramig, L. O. (2009). Suitability of dysphonia measurements for telemonitoring of parkinson's disease. IEEE Transactions on Biomedical Engineering, 56(4), 1015-1022. https://doi.org/10.1109/TBME.2008.2005954

Ma, A., Lau, K. K., & Thyagarajan, D. (2020). Voice changes in Parkinson’s disease: What are they telling us?. Journal of Clinical Neuroscience, 72, 1-7. https://doi.org/10.1016/j.jocn.2019.12.029